#Quantizing huggingface models to exl2
This version of my exl2 quantize colab creates a single quantizaion to download privatly.\
To calculate an estimate for VRAM size use: [NyxKrage/LLM-Model-VRAM-Calculator](https://huggingface.co/spaces/NyxKrage/LLM-Model-VRAM-Calculator)\
Not all models and architectures are compatible with exl2.\
Will upload to private hf repo in future.

In [ ]:
#@title Download and install environment
!git clone https://github.com/turboderp/exllamav2
%cd exllamav2
print("Installing pip dependencies")
!pip install -q -r requirements.txt
!pip install -q huggingface_hub requests tqdm
!pip install . -q
!wget https://raw.githubusercontent.com/oobabooga/text-generation-webui/main/download-model.py
modeldw = "none"

In [ ]:
#@title Login to HF (Required only for gated models)
#@markdown From my Colab/Kaggle login script on [Anthonyg5005/hf-scripts](https://huggingface.co/Anthonyg5005/hf-scripts/blob/main/HF%20Login%20Snippet%20Kaggle.py)
#import required functions
import os
from huggingface_hub import login, get_token, whoami

#get token
if os.environ.get('KAGGLE_KERNEL_RUN_TYPE', None) is not None: #check if user in kaggle
    from kaggle_secrets import UserSecretsClient
    from kaggle_web_client import BackendError
    try:
        login(UserSecretsClient().get_secret("HF_TOKEN")) #login if token secret found
    except BackendError:
        print('''
                      When using Kaggle, make sure to use the secret key HF_TOKEN.
                   This will prevent the need to login every time you run the script.
                   Set your secrets with the secrets add-on on the top of the screen.
             ''')
if get_token() is not None:
    #if the token is found then log in:
    login(get_token())
else:
    #if the token is not found then prompt user to provide it:
    login(input("API token not detected. Enter your HuggingFace (WRITE) token: "))

In [ ]:
#@title ##Choose HF model to download
#@markdown Weights must be stored in safetensors
if modeldw != "none":
    !rm {model}-{BPW}bpw.zip
    !rm -r {model}-exl2-{BPW}bpw
User = "meta-llama" # @param {type:"string"}
Repo = "Llama-2-7b-chat-hf" # @param {type:"string"}
modeldw = f"{User}/{Repo}"
model = f"{User}_{Repo}"
!python download-model.py {modeldw}

In [ ]:
#@title Quantize the model
#@markdown ###Quantization time will last based on model size
#@markdown Target bits per weight:
BPW = "4.125" # @param {type:"string"}
!mkdir {model}-exl2-{BPW}bpw-WD
!mkdir {model}-exl2-{BPW}bpw
!cp models/{model}/config.json {model}-exl2-{BPW}bpw-WD
#@markdown Calibrate with custom dataset, not recommended: (not finished on this version)
Calibrate = False # @param {type:"boolean"}
Calibrate = False
#@markdown Calibration dataset, check above (must be parquet file):
dataset = "wikitext" # @param {type:"string"}
if Calibrate == True:
    quant = f"convert.py -i models/{model} -o {model}-exl2-{BPW}bpw-WD -cf {model}-exl2-{BPW}bpw -c {dataset} -b {BPW}"
else:
    quant = f"convert.py -i models/{model} -o {model}-exl2-{BPW}bpw-WD -cf {model}-exl2-{BPW}bpw -b {BPW}"
!python {quant}

In [ ]:
#@title Zip and download the model
!rm -r {model}-exl2-{BPW}bpw-WD
!rm -r models/{model}
print("Zipping. May take a few minutes")
!zip -r {model}-{BPW}bpw.zip {model}-exl2-{BPW}bpw
from google.colab import files
files.download(f"{model}-{BPW}bpw.zip")
print("Colab download speeds very slow so download will take a while")